In [49]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
import os
import numpy as np
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

In [50]:
from chat_history_db import ChatHistory
import sqlite3
from datetime import datetime
import uuid
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [51]:
MODEL_NAME = "llama3.2"
llm = OllamaLLM(model= MODEL_NAME)

In [52]:
def load_docs():
    
    document_loader = []

    for root, dirs, files in os.walk("."):
        # Skip chroma_db folder
        if "faiss" in root or "git" in root:
            continue
        for file in files:
            if file.endswith(".pdf"):
                document_loader.append(file)

    return document_loader

In [53]:
document_loader = load_docs()
document_loader

['ENSC3016_Course_Notes_Part_1_Electromagnetism_Transformers.pdf',
 'ENSC3016_Course_Notes_Part_2_Electric_Machines.pdf',
 'Electric Machinery Fundamentals Textbook -- Chapman.pdf',
 'ENSC3016 Study Guide 1-Review of Circuit Fundamentals.pdf',
 'Three Phase Power System Fundamentals.pdf']

In [54]:
embedding_model ="sentence-transformers/all-MiniLM-L6-v2" #embedding matrix model

def embed_splitting(document_loader, embedding_model):
    embeddings = HuggingFaceEmbeddings(model = embedding_model, encode_kwargs={'normalize_embeddings': True})

    doc_store = []
    for file in document_loader:
        loader = PyPDFLoader(file)
        doc = loader.load()
        doc_store += doc

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size = 400,
        chunk_overlap = 64
        )
    
    #Make splits
    splits = text_splitter.split_documents(doc_store)

    return embeddings, splits


In [55]:
embeddings, splits = embed_splitting(document_loader, embedding_model)

In [56]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={'normalize_embeddings': True}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [57]:
len(splits)

402

In [58]:
dim = len(embeddings.embed_query("test sentence"))
index = faiss.IndexFlatL2(dim)

if os.path.exists("faiss_index"):
    print("Loading FAISS index from disk...")
    vector_store = FAISS.load_local("faiss_index", embeddings=embeddings, allow_dangerous_deserialization=True)
else:
    print("Building FAISS index from scratch...")
    dim = len(embeddings.embed_query("test sentence"))
    index = faiss.IndexFlatL2(dim)
    vector_store = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )
    vector_store.add_documents(splits)
    vector_store.save_local("faiss_index")

Loading FAISS index from disk...


In [59]:
# create the retriever object once
retriever = vector_store.as_retriever(search_kwargs={'k': 4})

In [60]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x31a378310>, search_kwargs={'k': 4})

In [ ]:
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [62]:
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x31a378310>, search_kwargs={'k': 4}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AI

In [63]:
qa_system_prompt = """You are an expert assistant answering based only on the provided context.

    Context:
    {context}
    
    Use all relevant information above to answer the question below. If the answer isn't found in the chunks, say:
    "I cannot answer this question because the necessary information was not found in the provided documents."

    When answering, cite the **source file name** and **slide/page number** if available.
    """

In [64]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [65]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [66]:
### Statefully manage chat history ###
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [69]:
conversational_rag_chain.invoke(
    {"input": "Explain transformers"},
    config={
        "configurable": {"session_id": "abc123"}
    },
)["answer"]

'Transformers are electrical devices that transfer electrical energy between two circuits through electromagnetic induction. They consist of a coil of wire, known as the primary coil, and another coil, known as the secondary coil, which is connected to a load.\n\nIn the context of the provided documents, transformers are discussed in relation to their application in electrical machines and systems. The auto-wound transformer is mentioned as an example of a step-down transformer with a ratio of 2:1, where the current in each winding is equal to half of the load current.\n\nThe variable transformer, which has a single-layer coil wound on a toroidal core, allows for the smooth variation of output voltage by moving a carbon brush along the surface of the coil. This is an application of the auto-wound transformer principle.\n\nIn 3-phase systems, transformers can be used in sets of three single-phase transformers or as three-set windings on three limbs of a core, as shown in figures 6-10 an

In [70]:
conversational_rag_chain.invoke(
    {"input": "Explain their properties?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Transformers have several key properties:\n\n1. **Magnetic Coupling**: Transformers use magnetic coupling to transfer energy between the primary and secondary coils. This is achieved by wrapping coils around a common magnetic core (Figure 6-6).\n2. **Electromagnetic Induction**: The primary coil induces a voltage in the secondary coil through electromagnetic induction, which is a fundamental principle of electromagnetism.\n3. **Ratio Control**: Transformers allow for ratio control, where the output voltage can be adjusted by changing the number of turns in the secondary coil relative to the primary coil.\n4. **Step-Down and Step-Up**: Transformers are used to step down or step up voltages, making it possible to transmit energy efficiently over long distances.\n5. **Toroidal Cores**: Toroidal transformers have a toroidal core, which is a ring-shaped core that allows for efficient magnetic coupling (Figure 6-5).\n6. **Shell-Type Construction**: The shell-type construction features coils